In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
df_train = pd.read_csv('../input/new-york-city-taxi-fare-prediction/train.csv', nrows=3000000)

In [3]:
df_train.head()

In [4]:
df_train.shape

In [5]:
df=df_train

In [6]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

In [7]:
df['year']= df['pickup_datetime'].dt.year

In [8]:
df['year'].value_counts()

In [9]:
# converting UTC to New York Timezone
df['newyork_time'] = df['pickup_datetime'] - pd.Timedelta(hours=4)

In [10]:
df['hour']= df['newyork_time'].dt.hour

In [ ]:
#df['pmam'] = df['hour'].apply(lambda x: 'am' if x <= 12 else 'pm')

In [11]:
df['pmam'] = df['hour'].apply(lambda x: '1' if x <= 12 else '2')

In [12]:
df['dayofweek'] = df['newyork_time'].dt.dayofweek

In [13]:
df['month'] = df['newyork_time'].dt.month

In [14]:
df.info()

In [15]:
df=df[df['fare_amount'] >0]

In [16]:
df['passenger_count'].value_counts()

In [17]:
df=df[df['passenger_count'] >=1]


In [18]:
df=df[df['passenger_count'] <=6]
df.info()

In [19]:
df['month'].value_counts().sort_values(ascending=True)

In [20]:
abs(df.corr()['fare_amount']).sort_values(ascending=False)

In [21]:
df.describe()
#newyork
#Latitude	40.730610
#Longitude	-73.935242

In [22]:
# hatalı lokasyon kayitlarini cikaralim.
df.query('pickup_longitude < -75' or 'pickup_latitude > 42' or 'dropoff_longitude < -75' or 'dropoff_latitude > 42')

In [23]:
df=df[df['pickup_longitude'] > -75]
df=df[df['pickup_latitude'] < 42]
df=df[df['dropoff_longitude'] > -75]
df=df[df['dropoff_latitude'] < 42]

In [24]:
df.reset_index(drop=True, inplace=True)
df

In [ ]:
#from geopy import distance
#df['distance_geopy'] = df.apply(lambda x: distance.distance((x['pickup_latitude'],x['pickup_longitude']), (x['dropoff_latitude'],x['dropoff_longitude'])).kilometres, axis=1)
# df.head()

In [25]:
def distance(df, lat1, long1, lat2, long2):
    # lldist: distance between lat-long coordinates
    r = 6371 # radius of the earth. Could be given in miles. DEFAULT: 6371 (km)
    phi1 = np.radians(df[lat1])
    phi2 = np.radians(df[lat2])
    
    delta_phi = np.radians(df[lat2] - df[lat1])
    delta_lambda = np.radians(df[long2] - df[long1])
    
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = (r * c)
    
    return d

In [26]:
df['distance'] = distance(df, 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude')

In [ ]:
# cols=['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'passenger_count','hour','pmam','dayofweek','year','month','distance']

In [ ]:
#for col in cols:
#    df[col] = df[col].astype('category')

In [ ]:
#df1=pd.get_dummies(df1,drop_first=True)
#df1.head()

In [27]:
df2=df.drop(['pickup_datetime','newyork_time', 'pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1)

In [28]:
df2.corr()

In [29]:
# x=df_train.iloc[:,0:8]
# x=df_train.drop(['key','fare_amount','pickup_datetime','NY_time','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count'],axis=1)
x=df.drop(['key','fare_amount','pickup_datetime','newyork_time', 'pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1)
#x=df.drop(['key','fare_amount'],axis=1)

In [30]:
y=df['fare_amount']

In [31]:
x = np.asarray(x).astype(np.float32)
y = np.asarray(y).astype(np.float32)

In [32]:
from keras.models import Sequential
from keras.layers import Dense

In [33]:
model=Sequential()
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [34]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [35]:
model.fit(x,y, epochs=20, batch_size=500, verbose=1)

In [36]:
import tensorflow as tf

In [37]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU

model = Sequential()
model.add(Conv2D(16, (3,3), padding='same', input_shape=(513, 25, 1)))
model.add(LeakyReLU())
model.add(Conv2D(16, (3,3), padding='same'))
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))
model.add(Conv2D(16, (3,3), padding='same'))
model.add(LeakyReLU())
model.add(Conv2D(16, (3,3), padding='same'))
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64))
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x,y, epochs=20, batch_size=500, verbose=1)